In [5]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from plotly.subplots import make_subplots
from scipy.interpolate import Rbf

In [2]:
df = pd.read_csv("d2012.csv")

In [ ]:
depth_mapping = {'Epipelagic': 2, 'Mesopelagic': 1, 'Bathypelagic':0}
df['Depth Zone Numeric'] = df['Depth Zone'].map(depth_mapping)

fig = px.scatter_3d(df, x='LATITUDE', y='LONGITUDE', z='Depth Zone Numeric', color='TEMP', text='Depth Zone', opacity=0.7)


fig.update_layout(scene=dict(xaxis_title='Latitude', yaxis_title='Longitude', zaxis_title='Depth Zone'))
fig.update_traces(marker_size = 3)
fig.show()

In [4]:
depth = "Epipelagic"
season = "Spring"
E_spring = df[(df['Depth Zone'] == depth) & (df['Season'] == season)]
E_spring = E_spring.groupby(['LATITUDE', 'LONGITUDE'])['TEMP'].mean().reset_index()

In [ ]:
X = E_spring[['LATITUDE', 'LONGITUDE']]
y = E_spring['TEMP']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, random_state=47)

gpr.fit(X_train, y_train)

y_pred = gpr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:
lat_long = [[latitude, longitude]]  # Replace latitude and longitude with test values
interpolated_temperature = gpr.predict(lat_long)
print(f'Interpolated temperature for coordinate ({latitude}, {longitude}): {interpolated_temperature[0]}')

In [ ]:
from scipy.interpolate import griddata

data = E_spring

lat, lon, temp = np.array(data).T

grid_lat, grid_lon = np.mgrid[min(lat):max(lat):100j, min(lon):max(lon):100j]

grid_temp = griddata((lat, lon), temp, (grid_lat, grid_lon), method='linear')

plt.figure()
plt.contourf(grid_lon, grid_lat, grid_temp)
plt.colorbar(label='Temperature')
plt.scatter(lon, lat, c=temp, edgecolors='k', marker='o', s=50)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Interpolated Temperature Contour Plot')
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Contour(x=grid_lon[0], y=grid_lat[:, 0], z=grid_temp, colorscale='Viridis'))

fig.add_trace(go.Scatter(x=lon, y=lat, mode='markers', marker=dict(size=10, color=temp, colorscale='Viridis', showscale=False, line=dict(
                color='MediumPurple',
                width=1
            ))))

fig.update_layout(
    title='Interpolated Temperature Contour Plot',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
)

fig.show()

In [17]:
data = E_spring

lat, lon, temp = np.array(data).T

# Fit the RBF model
rbf = Rbf(lat, lon, temp, function='multiquadric', smooth=1)


grid_lat, grid_lon = np.mgrid[min(lat)-1:max(lat)+1:100j, min(lon)-1:max(lon)+1:100j]

# Compute the interpolated and extrapolated values
grid_temp = rbf(grid_lat, grid_lon)


fig = go.Figure()

fig.add_trace(go.Contour(x=grid_lon[0], y=grid_lat[:, 0], z=grid_temp, colorscale='Viridis'))
fig.add_trace(go.Scatter(x=lon, y=lat, mode='markers', marker=dict(size=10, color=temp, colorscale='Viridis', showscale=False, line=dict(
                color='black',
                width=1
            ))))

fig.update_layout(
    title='Temperature Contour Plot : Epipelagic',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
)

fig.show()

In [13]:
depth = "Epipelagic"
season = "Spring"
E_spring = df[(df['Depth Zone'] == depth) & (df['Season'] == season)]
E_spring = E_spring.groupby(['LATITUDE', 'LONGITUDE'])['TEMP'].mean().reset_index()

In [14]:
depth = "Mesopelagic"
season = "Spring"
M_spring = df[(df['Depth Zone'] == depth) & (df['Season'] == season)][['LATITUDE', 'LONGITUDE', 'TEMP']]
M_spring = M_spring.groupby(['LATITUDE', 'LONGITUDE'])['TEMP'].mean().reset_index()

In [15]:
depth = "Bathypelagic"
season = "Spring"
B_spring = df[(df['Depth Zone'] == depth) & (df['Season'] == season)][['LATITUDE', 'LONGITUDE', 'TEMP']]
B_spring = B_spring.groupby(['LATITUDE', 'LONGITUDE'])['TEMP'].mean().reset_index()

In [19]:
def fit_rbf(data):
    lat, lon, temp = np.array(data).T
    rbf = Rbf(lat, lon, temp, function='multiquadric', smooth=1)
    grid_lat, grid_lon = np.mgrid[min(lat)-1:max(lat)+1:100j, min(lon)-1:max(lon)+1:100j]
    grid_temp = rbf(grid_lat, grid_lon)
    return lat, lon, temp, grid_lat, grid_lon, grid_temp

lat1, lon1, temp1, grid_lat1, grid_lon1, grid_temp1 = fit_rbf(E_spring)
lat2, lon2, temp2, grid_lat2, grid_lon2, grid_temp2 = fit_rbf(M_spring)
lat3, lon3, temp3, grid_lat3, grid_lon3, grid_temp3 = fit_rbf(B_spring)

fig = make_subplots(rows=1, cols=3, subplot_titles=("Epipelagic", "Mesopelagic", "Bathypelagic"))

contour_common_args = dict(showscale=True, coloraxis="coloraxis", contours=dict(start=0, end=10, size=1))

fig.add_trace(go.Contour(x=grid_lon1[0], y=grid_lat1[:, 0], z=grid_temp1, **contour_common_args), row=1, col=1)
fig.add_trace(go.Scatter(x=lon1, y=lat1, mode='markers', marker=dict(size=5, color=temp1, colorscale='Viridis', showscale=False, line=dict(color='black',width=1))), row=1, col=1)
fig.add_trace(go.Contour(x=grid_lon2[0], y=grid_lat2[:, 0], z=grid_temp2, **contour_common_args), row=1, col=2)
fig.add_trace(go.Scatter(x=lon2, y=lat2, mode='markers', marker=dict(size=5, color=temp2, colorscale='Viridis', showscale=False, line=dict(color='black',width=1))), row=1, col=2)
fig.add_trace(go.Contour(x=grid_lon3[0], y=grid_lat3[:, 0], z=grid_temp3, **contour_common_args), row=1, col=3)
fig.add_trace(go.Scatter(x=lon3, y=lat3, mode='markers', marker=dict(size=5, color=temp3, colorscale='Viridis', showscale=False, line=dict(color='black',width=1))), row=1, col=3)


fig.update_layout(coloraxis=dict(colorscale='Viridis', cmin=0, cmax=10))

fig.update_xaxes(title_text="Longitude", row=1, col=1)
fig.update_xaxes(title_text="Longitude", row=1, col=2)
fig.update_xaxes(title_text="Longitude", row=1, col=3)

fig.update_yaxes(title_text="Latitude", row=1, col=1)
fig.update_yaxes(title_text="Latitude", row=1, col=2)
fig.update_yaxes(title_text="Latitude", row=1, col=3)

fig.show()
